In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from sklearn import preprocessing
class DataFactory:
    def __init__(self, args, logger):
        self.args = args
        self.logger = logger
        self.home_dir = args.home_dir
        self.logger.info(f"current location: {os.getcwd()}")
        self.logger.info(f"home dir: {args.home_dir}")
        self.dataset_fn_dict = {
            "NeurIPS-TS-MUL": self.load_NeurIPS_TS_MUL,
            "SWaT": self.load_SWaT,
            "SMAP": self.load_SMAP,
            "MSL": self.load_MSL,
        }
        self.datasets = {
            "NeurIPS-TS-MUL": TSADStandardDataset,
            "SWaT": TSADStandardDataset,
            "SMAP": TSADStandardDataset,
            "MSL": TSADStandardDataset,
        }
        self.transforms = {
            "minmax": preprocessing.MinMaxScaler(),
            "std": preprocessing.StandardScaler(),
        }
    def __call__(self):
        self.logger.info(f"Preparing {self.args.dataset} ...")
        train_x, train_y, test_x, test_y = self.load()
        self.logger.info(
            f"train: X - {train_x.shape}, y - {train_y.shape} " +
            f"test: X - {test_x.shape}, y - {test_y.shape}"
        )
        self.logger.info(f"Complete.")
        self.logger.info(f"Preparing dataloader...")
        train_dataset, train_loader, test_dataset, test_loader = self.prepare(
            train_x, train_y, test_x, test_y,
            window_size=self.args.window_size,
            stride=self.args.stride,
            dataset_type=self.args.dataset,
            batch_size=self.args.batch_size,
            eval_batch_size=self.args.eval_batch_size,
            train_shuffle=True,
            test_shuffle=False,
            scaler=self.args.scaler,
            window_anomaly=self.args.window_anomaly
        )
        sample_X, sample_y = next(iter(train_loader))
        self.logger.info(f"total train dataset- {len(train_loader)}, "
                         f"batch_X - {sample_X.shape}, "
                         f"batch_y - {sample_y.shape}")
        sample_X, sample_y = next(iter(test_loader))
        self.logger.info(f"total test dataset- {len(test_loader)}, "
                         f"batch_X - {sample_X.shape}, "
                         f"batch_y - {sample_y.shape}")
        self.logger.info(f"Complete.")
        return train_dataset, train_loader, test_dataset, test_loader
    def load(self):
        return self.dataset_fn_dict[self.args.dataset](self.home_dir)
    def prepare(self, train_x, train_y, test_x, test_y,
                window_size,
                stride,
                dataset_type,
                batch_size,
                eval_batch_size,
                train_shuffle,
                test_shuffle,
                scaler,
                window_anomaly,
                ):
        transform = self.transforms[scaler]
        train_dataset = self.datasets[dataset_type](train_x, train_y,
                                                    flag="train", transform=transform,
                                                    window_size=window_size,
                                                    stride=stride,
                                                    window_anomaly=window_anomaly,
                                                    )
        train_dataloader = DataLoader(
            dataset=train_dataset,
            batch_size=batch_size,
            shuffle=train_shuffle,
        )
        transform = train_dataset.transform
        test_dataset = self.datasets[dataset_type](test_x, test_y,
                                                   flag="test", transform=transform,
                                                   window_size=window_size,
                                                   stride=stride,
                                                   window_anomaly=window_anomaly,
                                                   )
        test_dataloader = DataLoader(
            dataset=test_dataset,
            batch_size=eval_batch_size,
            shuffle=test_shuffle,
        )
        return train_dataset, train_dataloader, test_dataset, test_dataloader
    @staticmethod
    def load_NeurIPS_TS_MUL(home_dir="."):
        base_dir = "data/NeurIPS-TS"
        normal = pd.read_csv(os.path.join(home_dir, base_dir, "nts_mul_normal.csv"))
        abnormal = pd.read_csv(os.path.join(home_dir, base_dir, "nts_mul_abnormal.csv"))
        train_X, train_y = normal.values[:, :-1], normal.values[:, -1]
        test_X, test_y = abnormal.values[:, :-1], abnormal.values[:, -1]
        train_X, test_X = train_X.astype(np.float32), test_X.astype(np.float32)
        train_y, test_y = train_y.astype(int), test_y.astype(int)
        return train_X, train_y, test_X, test_y
    @staticmethod
    def load_SWaT(home_dir="."):
        base_dir = "data/SWaT"
        SWAT_TRAIN_PATH = os.path.join(home_dir, base_dir, 'SWaT_Dataset_Normal_v0.csv')
        SWAT_TEST_PATH = os.path.join(home_dir, base_dir, 'SWaT_Dataset_Attack_v0.csv')
        df_train = pd.read_csv(SWAT_TRAIN_PATH, index_col=0)
        df_test = pd.read_csv(SWAT_TEST_PATH, index_col=0)
        def process_df(df):
            for var_index in [item for item in df.columns if item != 'Normal/Attack']:
                df[var_index] = pd.to_numeric(df[var_index], errors='coerce')
            df.reset_index(drop=True, inplace=True)
            df.fillna(method='ffill', inplace=True)
            x = df.values[:, :-1].astype(np.float32)
            y = (df['Normal/Attack'] == 'Attack').to_numpy().astype(int)
            return x, y
        train_X, train_y = process_df(df_train)
        test_X, test_y = process_df(df_test)
        train_X, test_X = train_X.astype(np.float32), test_X.astype(np.float32)
        train_y, test_y = train_y.astype(int), test_y.astype(int)
        return train_X, train_y, test_X, test_y
    @staticmethod
    def load_SMAP(home_dir="."):
        base_dir = "data/SMAP"
        with open(os.path.join(home_dir, base_dir, "SMAP_train.pkl"), 'rb') as f:
            train_X = pickle.load(f)
        T, C = train_X.shape
        train_y = np.zeros((T,), dtype=int)
        with open(os.path.join(home_dir, base_dir, "SMAP_test.pkl"), 'rb') as f:
            test_X = pickle.load(f)
        with open(os.path.join(home_dir, base_dir, "SMAP_test_label.pkl"), 'rb') as f:
            test_y = pickle.load(f)
        train_X, test_X = train_X.astype(np.float32), test_X.astype(np.float32)
        train_y, test_y = train_y.astype(int), test_y.astype(int)
        return train_X, train_y, test_X, test_y
    @staticmethod
    def load_MSL(home_dir="."):
        base_dir = "data/MSL"
        with open(os.path.join(home_dir, base_dir, "MSL_train.pkl"), 'rb') as f:
            train_X = pickle.load(f)
        T, C = train_X.shape
        train_y = np.zeros((T,), dtype=int)
        with open(os.path.join(home_dir, base_dir, "MSL_test.pkl"), 'rb') as f:
            test_X = pickle.load(f)
        with open(os.path.join(home_dir, base_dir, "MSL_test_label.pkl"), 'rb') as f:
            test_y = pickle.load(f)
        train_X, test_X = train_X.astype(np.float32), test_X.astype(np.float32)
        train_y, test_y = train_y.astype(int), test_y.astype(int)
        return train_X, train_y, test_X, test_y
class TSADStandardDataset(Dataset):
    def __init__(self, x, y, flag, transform, window_size, stride, window_anomaly):
        super().__init__()
        self.transform = transform
        self.len = (x.shape[0] - window_size) // stride + 1
        self.window_size = window_size
        self.stride = stride
        self.window_anomaly = window_anomaly
        x, y = x[:self.len*self.window_size], y[:self.len*self.window_size]
        self.x = self.transform.fit_transform(x) if flag == "train" else self.transform.transform(x)
        self.y = y
    def __len__(self):
        return self.len
    def __getitem__(self, idx):
        _idx = idx * self.stride
        label = self.y[_idx:_idx+self.window_size]
        X, y = self.x[_idx:_idx+self.window_size], (1 in label) if self.window_anomaly else label
        return X, y
        return X, y

In [5]:
class THOC(nn.Module):
    def __init__(self, C, W, n_hidden, tau=1):
        self.L = math.floor(math.log(W, 2)) + 1 # number of DRNN layers
        self.DRNN = DRNN(n_input=C, n_hidden=n_hidden, n_layers=self.L)
        self.clusters = [nn.parameter(n_hidden, K_l) for K_l in range(self.L*2, 0, -2)]
        for c in self.clusters:
            nn.init.xavier_uniform_(c)
        self.tau = 1
        self.cnets = [
            nn.Sequential(
                nn.Linear(n_hidden, n_hidden),
                nn.ReLU(),
            ) for l in range(self.L) # nn that maps f_bar to f_hat
        ]
        for n in self.cnets:
            nn.init.xavier_uniform_(n)

        self.mlp = nn.Sequential(
            nn.Linear(n_hidden*2, n_hidden*2),
            nn.ReLU(),
            nn.Linear(n_hidden*2, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden)
        )

        self.TSSnets = [
            nn.parameter(W-2**l)
            for l in range(self.L)
        ]

    def forward(self, X):
        '''
        :param X: (B, L, C)
        :return: Losses,
        '''
        MTF_output = self.DRNN(X) # Multiscale Temporal Features from dilated RNN.

        # THOC
        L_THOC = 0
        f_t_bar = MTF_output[0][:, -1, :]
        Ps, Rs = [], []

        for i, c in enumerate(self.clusters):
            layer_output = F.cosine_similarity(f_t_bar, c)  # (B, num_cluster_{l-1}, hidden_dim) @ (hidden_dim, num_cluster_{l})
            P_ij = F.softmax(layer_output/self.tau, dim=-1) # (B, num_cluster_{l-1}, num_cluster_{l})
            R_ij = P_ij if len(Ps)==0 else Rs[i-1] @ P_ij
            Ps.append(P_ij)
            Rs.append(R_ij)

            c_vectors = self.cnets[i](f_t_bar) # (B, num_cluster_{l-1}, hidden_dim)
            f_t_bar = P_ij.transpose(-1, -2) @ c_vectors # (B, num_cluster_{l}, hidden_dim)
            if i != self.L-1:
                f_t_bar = self.MLP(torch.cat((f_t_bar, torch.repeat(MTF_output)[i + 1][:, -1, :].repeat(1, len(c), 1)), dim=-1))

            cosine_similarity = (f_t_bar / (torch.norm(f_t_bar, dim=-1) + 1e-08)) @ (c / (torch.norm(f_t_bar, dim=0) + 1e-08))
            d = 1 - cosine_similarity
            distances = R_ij * d
            L_THOC += torch.mean(distances)

        # ORTH
        L_orth = 0
        for c in self.clusters:
            c_sq = c.T @ c #(K_l, K_l)
            K_l, _ = c_sq.shape
            L_orth += torch.linalg.matrix_norm(c_sq-torch.eye(K_l))
        L_orth /= len(self.clusters)

        # TSS
        L_TSS = 0
        for i, net in enumerate(self.TSSnets):
            src, tgt = X[:, :2**(i), :], X[:, -2**(i):, :]
            L_TSS += F.mse_loss(src*net, tgt)

        return {
            "L_THOC": L_THOC,
            "L_orth": L_orth,
            "L_TSS": L_TSS
        }

# Dilated RNN code modified from:
# https://github.com/zalandoresearch/pytorch-dilated-rnn/blob/master/drnn.py
class DRNN(nn.Module):
    def __init__(self, n_input, n_hidden, n_layers, dropout=0, cell_type='GRU', batch_first=True):
        super(DRNN, self).__init__()

        self.dilations = [2 ** i for i in range(n_layers)]
        self.cell_type = cell_type
        self.batch_first = batch_first

        layers = []
        if self.cell_type == "GRU":
            cell = nn.GRU
        elif self.cell_type == "RNN":
            cell = nn.RNN
        elif self.cell_type == "LSTM":
            cell = nn.LSTM
        else:
            raise NotImplementedError

        for i in range(n_layers):
            if i == 0:
                c = cell(n_input, n_hidden, dropout=dropout)
            else:
                c = cell(n_hidden, n_hidden, dropout=dropout)
            layers.append(c)
        self.cells = nn.Sequential(*layers)

    def forward(self, inputs, hidden=None):
        inputs = inputs.transpose(0, 1) if self.batch_first else inputs
        outputs = []
        for i, (cell, dilation) in enumerate(zip(self.cells, self.dilations)):
            if hidden is None:
                inputs, _ = self.drnn_layer(cell, inputs, dilation)
            else:
                inputs, hidden[i] = self.drnn_layer(cell, inputs, dilation, hidden[i])
            _output = inputs.transpose(0, 1) if self.batch_first else inputs
            outputs.append(_output)
        return outputs

    def drnn_layer(self, cell, inputs, rate, hidden=None):
        n_steps = len(inputs)
        batch_size = inputs[0].size(0)
        hidden_size = cell.hidden_size

        inputs, _ = self._pad_inputs(inputs, n_steps, rate)
        dilated_inputs = self._prepare_inputs(inputs, rate)

        if hidden is None:
            dilated_outputs, hidden = self._apply_cell(dilated_inputs, cell, batch_size, rate, hidden_size)
        else:
            hidden = self._prepare_inputs(hidden, rate)
            dilated_outputs, hidden = self._apply_cell(dilated_inputs, cell, batch_size, rate, hidden_size, hidden=hidden)

        splitted_outputs = self._split_outputs(dilated_outputs, rate)
        outputs = self._unpad_outputs(splitted_outputs, n_steps)

        return outputs, hidden

    def _apply_cell(self, dilated_inputs, cell, batch_size, rate, hidden_size, hidden=None):
        if hidden is None:
            if self.cell_type == 'LSTM':
                c, m = self.init_hidden(batch_size * rate, hidden_size)
                hidden = (c.unsqueeze(0), m.unsqueeze(0))
            else:
                hidden = self.init_hidden(batch_size * rate, hidden_size).unsqueeze(0)

        dilated_outputs, hidden = cell(dilated_inputs, hidden)

        return dilated_outputs, hidden

    def _unpad_outputs(self, splitted_outputs, n_steps):
        return splitted_outputs[:n_steps]

    def _split_outputs(self, dilated_outputs, rate):
        batchsize = dilated_outputs.size(1) // rate

        blocks = [dilated_outputs[:, i * batchsize: (i + 1) * batchsize, :] for i in range(rate)]

        interleaved = torch.stack((blocks)).transpose(1, 0).contiguous()
        interleaved = interleaved.view(dilated_outputs.size(0) * rate,
                                       batchsize,
                                       dilated_outputs.size(2))
        return interleaved

    def _pad_inputs(self, inputs, n_steps, rate):
        is_even = (n_steps % rate) == 0

        if not is_even:
            dilated_steps = n_steps // rate + 1

            zeros_ = torch.zeros(dilated_steps * rate - inputs.size(0),
                                 inputs.size(1),
                                 inputs.size(2))

            inputs = torch.cat((inputs, zeros_))
        else:
            dilated_steps = n_steps // rate

        return inputs, dilated_steps

    def _prepare_inputs(self, inputs, rate):
        dilated_inputs = torch.cat([inputs[j::rate, :, :] for j in range(rate)], 1)
        return dilated_inputs

    def init_hidden(self, batch_size, hidden_dim):
        hidden = torch.zeros(batch_size, hidden_dim)
        if self.cell_type == "LSTM":
            memory = torch.zeros(batch_size, hidden_dim)
            return (hidden, memory)
        else:
            return hidden

if __name__ == "__main__":
    B, L, C = 64, 12, 51
    ip = torch.randn((B, L, C))
    drnn = DRNN(n_input=C, n_hidden=128, n_layers=3, dropout=0, cell_type='GRU', batch_first=True)

    print(drnn)
    print(drnn(ip))

DRNN(
  (cells): Sequential(
    (0): GRU(51, 128)
    (1): GRU(128, 128)
    (2): GRU(128, 128)
  )
)
[tensor([[[ 2.4335e-01,  7.4354e-02,  2.6042e-01,  ..., -1.2665e-01,
           3.9342e-01, -1.0363e-01],
         [ 1.8304e-01, -5.1456e-02,  1.8631e-01,  ..., -6.3175e-02,
           3.6321e-01, -2.2124e-01],
         [ 2.8701e-01,  2.1541e-01,  3.3847e-01,  ..., -7.1447e-02,
           4.7187e-02,  8.9042e-02],
         ...,
         [-9.5338e-02,  1.7424e-01, -7.5130e-04,  ...,  1.2272e-02,
          -2.2933e-01,  1.8912e-01],
         [-1.0974e-01,  7.3719e-02, -6.6230e-02,  ...,  1.2789e-01,
          -4.0509e-01,  3.7038e-02],
         [ 1.9439e-01,  1.1886e-01,  7.9913e-02,  ...,  2.3062e-01,
          -1.5337e-01, -3.3471e-01]],

        [[-1.2886e-01, -9.7856e-02,  6.8023e-02,  ...,  5.1745e-02,
           1.5321e-02, -6.5128e-02],
         [-2.3255e-02,  1.6872e-01,  6.4116e-02,  ..., -4.9134e-02,
          -8.4108e-02,  8.8218e-02],
         [ 1.5735e-01,  2.9138e-01,  1.6